In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# fetchall must be called after running execute to get results of query need iterate over fetch all to get data you want
# list of tuples 
#[(,,,),... get idea]

In [34]:
# Establishing a connectionpool

from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector.errors import Error

user = os.getenv('USERNAME')
pw = os.getenv('PASSWORD')

dbconfig={"database":"little_lemon_db", "user":user, "password":pw}
print (dbconfig['user'])

# Creating connection pool with two connections
try:
    pool_a = MySQLConnectionPool(
        pool_name='little_lemon_pool',
        pool_size=2,
        host='localhost',
        database='little_lemon_db',
        user=user,
        password=pw)
    print("Connection pool 'pool_a' created successfully.")
except Error as e:
    print(f"An error occurred while creating the connection pool: {e}")

# Obtain connection from pool_a create cursor object to communicate wiht db

try:
    connection = pool_a.get_connection()
    print("connection obtained from pool")
    if connection.is_connected():
        print('active connection')
        cursor = connection.cursor()
        print('cursor obj created')
except Error as e:
    print(f"An error occurred while obtaining a connection or executing a query: {e}")    

adanglol
Connection pool 'pool_a' created successfully.
connection obtained from pool
active connection
cursor obj created


In [ ]:
# Now that we created cursor now we implement stored procedure called PeakHours to check peak hours for little lemonn
# As well as for guest status 

peak_hours_proc = """
CREATE PROCEDURE PeakHours()
BEGIN
    SELECT 
        HOUR(BookingSlot) AS BookingHour,
        COUNT(*) AS NumberOfBookings
    FROM bookings
    GROUP BY
        BookingHour
    ORDER BY
        NumberOfBookings DESC;
END
"""


guest_status_proc="""
CREATE PROCEDURE GuestStatus()
BEGIN
    SELECT 
        CONCAT(GuestFirstName,' ',GuestLastName) AS FullName,
        CASE
            WHEN e.Role IN ('Manager','Assistant Manager') THEN 'Ready to pay'
            WHEN e.Role = 'Head Chef' THEN 'Ready to serve'
            WHEN e.Role = 'Assistant Chef' THEN 'Preparing Order'
            WHEN e.Role = 'Head Waiter' THEN 'Order served'
            ELSE 'Unknown Status'
        END AS OrderStatus
    FROM 
        bookings b
    LEFT JOIN
        employees e ON b.EmployeeID = e.EmployeeID;
END
"""

In [ ]:
# TO ADD OUR PROCEDURES TO OUR DB
try:
    cursor.execute(peak_hours_proc)
    cursor.execute(guest_status_proc)
    print('Peak Hours and Guest Status procs been added successsfully!')
except Error as e:
    print(f"An error occured creating procedures: {e}")

In [35]:
# CALL PROCEDURES AND PROCESS DATA
try:
    # Call the first stored procedure (PeakHours)
    cursor.execute("CALL PeakHours();")
    
    # Fetch the result of the first stored procedure
    peak_hours_data = cursor.fetchall()
    
    print("Peak Hours Data:")
    # Loop through each record from PeakHours result
    for row in peak_hours_data:
        # Assuming the result contains two columns: BookingHour and NumberOfBookings
        print(f"Booking Hour: {row[0]}, Number of Bookings: {row[1]}")
        
    # Explicitly close the first result set to ensure synchronization
    while cursor.nextset():
        pass  # Close any additional result sets if present

    # Call the second stored procedure (GuestStatus)
    cursor.execute("CALL GuestStatus();")
    
    # Fetch the result of the second stored procedure
    guest_status_data = cursor.fetchall()
    
    print("Guest Status Data:")
    # Loop through each record from GuestStatus result
    for row in guest_status_data:
        # Assuming the result contains the guest's name and status
        print(f"Guest: {row[0]}, Status: {row[1]}")

    print('Successfully called procs and accessed data')
except Error as e:
    print(f"An error occurred while calling the procedures: {e}")

Peak Hours Data:
Booking Hour: 19, Number of Bookings: 2
Booking Hour: 15, Number of Bookings: 1
Booking Hour: 17, Number of Bookings: 1
Booking Hour: 18, Number of Bookings: 1
Booking Hour: 20, Number of Bookings: 1
Guest Status Data:
Guest: Anna Iversen, Status: Ready to pay
Guest: Joakim Iversen, Status: Ready to pay
Guest: Vanessa McCarthy, Status: Ready to serve
Guest: Marcos Romero, Status: Preparing Order
Guest: Hiroki Yamane, Status: Ready to pay
Guest: Diana Pinto, Status: Order served
Successfully called procs and accessed data
